# Data preprocessing pipeline
Note: we won't cover this at all in class; this is just so you can see the steps that went into produce the dataset we'll use.

In [47]:
import pandas as pd
import numpy as np

# Disable annoying SettingWithCopyWarning
pd.options.mode.chained_assignment = None

### Load outcome data

In [48]:
data = pd.read_csv('../data/Austin_Animal_Center_Outcomes.csv')

### Rename columns for brevity

In [49]:
replacements = {
    'Animal ID': 'id',
    'DateTime': 'datetime',
    'Outcome Type': 'outcome',
    'Outcome Subtype': 'outcome_subtype',
    'Animal Type': 'animal',
    # We append these with '_string' because we're going
    # to use the names 'sex' and 'age' later for
    # extracted variables, so this avoids confusion.
    'Sex upon Outcome': 'sex_string',
    'Age upon Outcome': 'age_string',
}

# # We can apply the rename function to either row names
# # or column names. Here we're doing columns.
data = data.rename(columns=replacements)

# # Let's also make sure all column names are in lowercase.
# # We're using an idiom called a "list comprehension".
data.columns = [c.lower() for c in data.columns]

### Convert timestamp to datetime and extract time variables

In [50]:
# Convert the datetime column to actual datetimes
data['datetime'] = pd.to_datetime(data['datetime'])

# Datetime-related functions are accessed through the .dt attribute
data['day'] = data['datetime'].dt.weekday
data['hour'] = data['datetime'].dt.hour
data['month'] = data['datetime'].dt.month
data['year'] = data['datetime'].dt.year

### Extract age in months

In [51]:
def extract_months(value):
    ''' Extract and return age in months from string values 
    "3 years" or "7 months".
    
    Note: this code won't work as written, because some of
    the values passed in are invalid.
    
    Exercise: Modify the code to return the special value
    np.nan (numpy's representation of not-a-number)
    whenever an invalid value is encountered.
    '''
    try:
        number, unit = value.split(' ')
        number = int(number)

        if unit.startswith('year'):
            number *= 12
        elif unit.startswith('week'):
            number *= 0.25
    except:
        number = np.nan

    return number

# Convert the age string to a numeric variable (no. of months)
data['age'] = data['age_string'].apply(extract_months)

### Recode sex string as binary indicators for sex and sterilization

In [52]:
# Replace the sole NaN in sex_string with 'Unknown'
data['sex_string'] = data['sex_string'].fillna('Unknown')

# Initialize a new column with NaN as the default value (for unknown sex)
data['sex'] = np.nan

# Update the column with values for M (0) and F (1)
inds = data['sex_string'].str.contains('Male')
data['sex'][inds] = 0
data['sex'][data['sex_string'].str.contains('Female')] = 1

# Now do the same kind of thing for sterilization
data['sterilized'] = 1
data['sterilized'][data['sex_string'].str.contains('Intact')] = 0
data['sterilized'][data['sex_string'].str.contains('Unknown')] = np.nan

### Keep only the first color in cases of mixed color

In [53]:
# pandas string methods are accessed through a column's str attribute

# store the first color in the DataFrame
data['first_color'] = data['color'].str.split('/').str.get(0)

# also store an indicator for mixed/single color
data['mixed_color'] = data['color'].str.count('/')

### Keep only the last outcome for each unique animal

In [54]:
# Sort all outcomes by chronological order.
data = data.sort_values('datetime')

# Keep only the last outcome for each unique animal
data = data.groupby('id').last().reset_index()

### Merge in scraped breed data

In [55]:
# Load the breed data to merge in
breed_data = pd.read_csv('../data/breed_data.csv')

# Let's prepare a 'merge_key' column in our outcome data that should line up
# as well as possible with the breed_name column in the breed_data DataFrame.
# We'll start by dropping all occurrences of 'Mix', 'Shorthair', etc.
data['merge_key'] = data['breed'].replace([' Mix', ' Shorthair', ' Longhair'], '', regex=True)

# Next, we'll keep only the first breed in cases where there are multiple given.
# For example, "Beagle/Labrador Retriever" would become just "Beagle".
data['merge_key'] = data['merge_key'].str.split('/').str.get(0)

# Manually map the most common breeds that don't align. I've cheated here
# and done a bit of extra work ahead of time to figure out what these are.
replacements = {
    "Pit Bull": "American Staffordshire Terrier",
    "Staffordshire": "Staffordshire Bull Terrier",
    "German Shepherd": "German Shepherd Dog",
    "Anatol Shepherd": "Anatolian Shepherd Dog",
    "Australian Shepherd": "Australian Shepherd Dog",
    "Catahoula": "Catahoula Leopard Dog",
    "American Pit Bull Terrier": "American Staffordshire Terrier", 
    "Doberman Pinsch": "Doberman Pinscher",  
}

# Apply the replacements
data['merge_key'] = data['merge_key'].replace(replacements)

# Finally, we're in a position to merge our data.
# We have to tell pandas which columns to use in each of the two datasets.
# We can also specify how we want the join to work ('left', 'outer',
# 'inner', etc.). The default is left, meaning that the output DataFrame
# will have the same number of rows as the first input DataFrame (data).
data = data.merge(breed_data, left_on='merge_key', right_on='breed_name', how='left')

### Save to file

In [56]:
data.to_csv('../data/preprocessed_data.csv', index=False, encoding='utf-8')